<h1 align=center><font size = 5>Capstone Project - The Battle of Neighborhoods (Week 1-2)</font></h1>

## Business Problem Section

#### Background

According to "Business Time", Singapore property market expected to hold steady in 2020 will continue to be a key market for global investors in 2020.

#### Business Problem

In my case, what I want to do is to find out the undervalued properties in Singapore market and find the Nearby Venues and explore the undervalued properties with their relavant features and provide useful information to my investors.

## Data section

The SG property data is crawled from (https://www.edgeprop.sg/). I first crawled all the property data from the website and only filtered the properties who is on sale and has fairvalue (which is the undervalued properties.)

I used another seperate notebook to crawl the data and generate a clean dataset which I used for my model training the map plotting. In each row, I provided the URL which directly link to the certain property which you can use it for your reference. For the timing saving purpose, I won't put the code in this notebook and run it with model. 

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Import and Explore Dataset</a>

2. <a href="#item2">Explore Neighborhoods in SG</a>

3. <a href="#item3">Analyze Each Neighborhood</a>

4. <a href="#item4">Cluster Neighborhoods</a>

5. <a href="#item5">Examine Clusters</a> 

6. <a href="#item6">Conclusions</a>
</font>
</div>

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from onemapsg import OneMapClient

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<a id='item1'></a>

## 1. Import and Explore Dataset

In [20]:
with open('P1.csv',errors='ignore') as f:
    SG_property = pd.read_csv(f)
    SG_property['Postcode']=SG_property['Postcode'].replace('', np.nan)
    SG_property['Price']=SG_property['Price'].replace('',np.nan)
    SG_property=SG_property.dropna(subset=['Price','Postcode'])
print(len(SG_property))

569


In [21]:
SG_property['Postcode']=SG_property['Postcode'].str.strip(',')
SG_property['Postcode']=SG_property['Postcode'].astype('str').str.strip('.0')
SG_property = SG_property.groupby(['Name','Postcode'], as_index=False, sort=False)['Price'].mean()

In [22]:
SG_property.head()

,Name,Postcode,Price
0,THE CREST,159017,1.788000e+06
1,CARIBBEAN AT KEPPEL BAY,98657,2.999999e+06
2,RIVERFRONT RESIDENCES,530344,9.477000e+05
3,TREASURE AT TAMPINES,520118,1.089190e+06
4,Parc Clematis,128793,1.491167e+06


In [23]:
SG_property.shape

(224, 3)

There are 224 unique Property names in the above dataset

### Find the coordinates for each Postcode

In [24]:
Client = OneMapClient.OneMapClient("pattygaohua@gmail.com", "Lihua3865038")

longitude=[]
latitude=[]
Name=[]
Postcode=[]
Price=[]

i=0

for id in SG_property['Postcode']:
    try:
        res = Client.search(id)
        latitude.append(float(res['results'][0]['LATITUDE']))
        longitude.append(float(Client.search(id)['results'][0]['LONGITUDE']))
        Name.append(SG_property.iloc[i]['Name'])
        Postcode.append(SG_property.iloc[i]['Postcode'])
        Price.append(SG_property.iloc[i]['Price'])
        i+=1
    except:
#         print(id)
        pass

In [25]:
SG_property = pd.DataFrame()
SG_property['Name']=Name
SG_property['Postcode']=Postcode
SG_property['Price']=Price
SG_property['longitude']=longitude
SG_property['latitude']=latitude

SG_property.head()

,Name,Postcode,Price,longitude,latitude
0,THE CREST,159017,1.788000e+06,103.820411,1.292695
1,CARIBBEAN AT KEPPEL BAY,98657,2.999999e+06,103.758984,1.317181
2,RIVERFRONT RESIDENCES,530344,9.477000e+05,103.897730,1.410047
3,TREASURE AT TAMPINES,520118,1.089190e+06,103.873225,1.365660
4,Parc Clematis,128793,1.491167e+06,103.855776,1.346814


#### Use geopy library to get the latitude and longitude values of Singapore.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

In [26]:
address = 'Singapore'

geolocator = Nominatim(user_agent="SG_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Singapore are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Singapore are 1.357107, 103.8194992.


#### Create a map of SG with Property Name, Property average Price superimposed on top
you can zoom in/out in the below plot and click on to find the property name and price

In [42]:
# create map of SG using latitude and longitude values
map_SG = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for latitude, longitude, Name, Postcode, Price in zip(SG_property['latitude'], 
                                           SG_property['longitude'], 
                                           SG_property['Name'], 
                                           SG_property['Postcode'],
                                           SG_property['Price']):
    label = '{}, {}'.format(Name, Price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.1,
        parse_html=False).add_to(map_SG)  
    
map_SG

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [43]:
CLIENT_ID = '1SOXVIFBLAJKBSWTQEMBFAWMJ5SX1P1PU2B1NZUO1SIRIGCO' # your Foursquare ID
CLIENT_SECRET = 'NQ1D43DPFC2ZLMN0NCSXR1WAOZX1O4SGRPXCKD3FEQBC2SGK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1SOXVIFBLAJKBSWTQEMBFAWMJ5SX1P1PU2B1NZUO1SIRIGCO
CLIENT_SECRET:NQ1D43DPFC2ZLMN0NCSXR1WAOZX1O4SGRPXCKD3FEQBC2SGK


#### Now, let's get the top 100 venues that are in center of SG within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [44]:
# type your answer here
LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
CLIENT_ID, CLIENT_SECRET, VERSION, latitude, longitude, radius, LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=1SOXVIFBLAJKBSWTQEMBFAWMJ5SX1P1PU2B1NZUO1SIRIGCO&client_secret=NQ1D43DPFC2ZLMN0NCSXR1WAOZX1O4SGRPXCKD3FEQBC2SGK&v=20180605&ll=1.40059849499751,103.898920186089&radius=500&limit=100'

Send the GET request and examine the resutls

In [52]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e1d6a2b40a7ea001c57911b'},
 'response': {'headerLocation': 'Punggol',
  'headerFullLocation': 'Punggol, Singapore',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 1.4050984994975146,
    'lng': 103.90341313090252},
   'sw': {'lat': 1.3960984904975056, 'lng': 103.89442724127548}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5683dd3e498e232d49ceabb6',
       'name': 'North Waterfall',
       'location': {'lat': 1.4027992538246539,
        'lng': 103.89708143411224,
        'labeledLatLngs': [{'label': 'display',
          'lat': 1.4027992538246539,
          'lng': 103.89708143411224}],
        'distance': 319,
        'cc': 'SG',
        'neighborhood': 'Punggol',
        'c

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [46]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [47]:
venues = results['response']['groups'][0]['items']
 
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,North Waterfall,Pool,1.402799,103.897081
1,Twin Waterfalls Gym,Gym,1.402764,103.896747
2,Bus stop 65149 (Soo Teck Stn),Bus Station,1.403399,103.896747
3,TasteBud,Food Court,1.403221,103.895791


And how many venues were returned by Foursquare?

In [48]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


<a id='item2'></a>

## 2. Explore Each Property Region in SG
(we will call each property region as Neighborhood below)

In [55]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now ;et's create a new dataframe called *SG_venues*.

In [56]:
SG_venues = getNearbyVenues(names=SG_property['Name'],
                                   latitudes=SG_property['latitude'],
                                   longitudes=SG_property['longitude']
                                  )

THE CREST
CARIBBEAN AT KEPPEL BAY
RIVERFRONT RESIDENCES
TREASURE AT TAMPINES
Parc Clematis
PARC ESTA
PIERMONT GRAND
AFFINITY AT SERANGOON
159 Bishan Street 13
565 Ang Mo Kio Avenue 3
PARK COLONIAL
JADESCAPE
WILSHIRE RESIDENCES
FOURTH AVENUE RESIDENCES
THE FLORENCE RESIDENCES
REFLECTIONS AT KEPPEL BAY
WILSHIRE RESIDENCES
REFLECTIONS AT KEPPEL BAY
THE TRE VER
Avenue South Residence
ONE-NORTH RESIDENCES
PARKVIEW APARTMENTS
REGENT HEIGHTS
MARINA ONE RESIDENCES
AFFINITY AT SERANGOON
Pullman Residences
Neu at Novena
JADESCAPE
WILSHIRE RESIDENCES
226 Bishan Street 23
SPRING GROVE
PARC ESTA
VIEW AT KISMIS
JADESCAPE
BIJOU
DOUBLE BAY RESIDENCES
59 Strathmore Avenue
PARKVIEW APARTMENTS
FOURTH AVENUE RESIDENCES
MAYFAIR MODERN
FOURTH AVENUE RESIDENCES
35 GILSTEAD
JUNIPER HILL
FYVE DERBYSHIRE
AMBER PARK
PALMERA RESIDENCE
NEW FUTURA
8 SAINT THOMAS
Seaside Residences
BOULEVARD VUE
METRO LOFT
NEW FUTURA
JUI RESIDENCES
CONCOURSE SKYLINE
TREASURE AT TAMPINES
PIERMONT GRAND
DAINTREE RESIDENCE
ONE HOLLAND 

#### Let's check the size of the resulting dataframe

In [57]:
print(SG_venues.shape)
SG_venues.head()

(2967, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,THE CREST,1.292695,103.820411,Alexandra Canal,1.292071,103.822217,Canal
1,THE CREST,1.292695,103.820411,Delta Swimming Complex,1.289225,103.819930,Pool
2,THE CREST,1.292695,103.820411,ActiveSG Gym (Delta Sports Complex),1.290473,103.820374,Gym
3,THE CREST,1.292695,103.820411,102 Henderson Crescent,1.290089,103.821613,Coffee Shop
4,THE CREST,1.292695,103.820411,Bus Stop 10279 (Tanglin View),1.291413,103.817161,Bus Station


Let's check how many venues were returned for each neighborhood

In [58]:
SG_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
134 Bishan Street 12,23,23,23,23,23,23
141 Marsiling Road,15,15,15,15,15,15
159 Bishan Street 13,5,5,5,5,5,5
189A Rivervale Drive,11,11,11,11,11,11
207B Punggol Place,4,4,4,4,4,4
226 Bishan Street 23,7,7,7,7,7,7
259C Punggol Field,4,4,4,4,4,4
273B Bishan Street 24,1,1,1,1,1,1
275 Toh Guan Road,11,11,11,11,11,11


#### Let's find out how many unique categories can be curated from all the returned venues

In [59]:
print('There are {} uniques categories.'.format(len(SG_venues['Venue Category'].unique())))

There are 260 uniques categories.


<a id='item3'></a>

## 3. Analyze Each Neighborhood

In [60]:
# one hot encoding
SG_onehot = pd.get_dummies(SG_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
SG_onehot['Neighborhood'] = SG_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [SG_onehot.columns[-1]] + list(SG_onehot.columns[:-1])
SG_onehot = SG_onehot[fixed_columns]

SG_onehot.head()

,Yoga Studio,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Beach,Beach Bar,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Cafeteria,Café,Campground,Canal,Cantonese Restaurant,Casino,Chinese Aristocrat Restaurant,Chinese Breakfast Place,Chinese Restaurant,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Cafeteria,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Electronics Store,English Restaurant,Event Space,Exhibit,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Fujian Restaurant,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Halal Restaurant,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laundromat,Light Rail Station,Lighthouse,Lingerie Store,Lottery Retailer,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Metro Station,Mexican Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Venue,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parking,Pedestrian Plaza,Peking Duck Restaurant,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pier,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,River,Road,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Skate Park,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Tapas Restaurant,Taxi,Taxi Stand,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Train Station,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Watch Shop,Water Park,Waterfront,Whisky Bar,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

And let's examine the new dataframe size.

In [61]:
SG_onehot.shape

(2967, 260)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [62]:
SG_grouped = SG_onehot.groupby('Neighborhood').mean().reset_index()
SG_grouped

,Neighborhood,Yoga Studio,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Beach,Beach Bar,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Cafeteria,Café,Campground,Canal,Cantonese Restaurant,Casino,Chinese Aristocrat Restaurant,Chinese Breakfast Place,Chinese Restaurant,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Cafeteria,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Electronics Store,English Restaurant,Event Space,Exhibit,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Fujian Restaurant,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Halal Restaurant,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laundromat,Light Rail Station,Lighthouse,Lingerie Store,Lottery Retailer,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Metro Station,Mexican Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Venue,Nature Preserve,New American Restaurant,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parking,Pedestrian Plaza,Peking Duck Restaurant,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pier,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,River,Road,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Skate Park,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Tapas Restaurant,Taxi,Taxi Stand,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Train Station,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Watch Shop,Water Park,Waterfront,Whisky Bar,Wine Bar,Wings Joint,Women's Store
0,134 Bishan Street 12,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.086957,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0

#### Let's confirm the new size

In [63]:
SG_grouped.shape

(122, 260)

In [64]:
SG_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Beach,Beach Bar,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Cafeteria,Café,Campground,Canal,Cantonese Restaurant,Casino,Chinese Aristocrat Restaurant,Chinese Breakfast Place,Chinese Restaurant,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Cafeteria,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Electronics Store,English Restaurant,Event Space,Exhibit,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Fujian Restaurant,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Halal Restaurant,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laundromat,Light Rail Station,Lighthouse,Lingerie Store,Lottery Retailer,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Metro Station,Mexican Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Venue,Nature Preserve,New American Restaurant,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parking,Pedestrian Plaza,Peking Duck Restaurant,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pier,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,River,Road,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Skate Park,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Tapas Restaurant,Taxi,Taxi Stand,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Train Station,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Watch Shop,Water Park,Waterfront,Whisky Bar,Wine Bar,Wings Joint,Women's Store
0,134 Bishan Street 12,0.0,0.0,0.0,0.0,0.0,0.0,0.043478,0.0,0.0,0.000000,0.0,0.0,0.086957,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.043478,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.173913,0.0,0.0,0.0,0.0,0.0,0.0,0.130435,0.0,0

#### Let's print each neighborhood along with the top 5 most common venues

In [65]:
num_top_venues = 5

for hood in SG_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = SG_grouped[SG_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:] # we take the 2nd rows onwards
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----134 Bishan Street 12----
                  venue  freq
0    Chinese Restaurant  0.17
1          Noodle House  0.13
2           Coffee Shop  0.13
3                Bakery  0.09
4  Fast Food Restaurant  0.09


----141 Marsiling Road----
               venue  freq
0             Bakery  0.07
1          BBQ Joint  0.07
2           Tea Room  0.07
3  Hainan Restaurant  0.07
4        Coffee Shop  0.07


----159 Bishan Street 13----
          venue  freq
0      Bus Stop   0.4
1          Park   0.2
2   Pizza Place   0.2
3         Trail   0.2
4  Noodle House   0.0


----189A Rivervale Drive----
                venue  freq
0            Bus Stop  0.18
1  Chinese Restaurant  0.09
2                Café  0.09
3          Food Court  0.09
4    Asian Restaurant  0.09


----207B Punggol Place----
               venue  freq
0         Playground  0.75
1  Convenience Store  0.25
2        Yoga Studio  0.00
3       Noodle House  0.00
4  Other Repair Shop  0.00


----226 Bishan Street 23----
                

                    venue  freq
0             Bus Station  0.29
1  Thrift / Vintage Store  0.14
2                    Pool  0.14
3                Bus Stop  0.14
4           Jewelry Store  0.14


----EIGHT RIVERSUITES----
              venue  freq
0       Coffee Shop  0.12
1        Food Court  0.12
2  Asian Restaurant  0.06
3       Bus Station  0.03
4        Soup Place  0.03


----FLORAVILLE----
               venue  freq
0   Sculpture Garden  0.33
1       Burger Joint  0.33
2         Playground  0.33
3       Noodle House  0.00
4  Other Repair Shop  0.00


----FORESQUE RESIDENCES----
                venue  freq
0  Seafood Restaurant  0.09
1    Malay Restaurant  0.09
2                Pool  0.09
3             Stadium  0.09
4    Asian Restaurant  0.09


----FOURTH AVENUE RESIDENCES----
                 venue  freq
0  Japanese Restaurant  0.09
1                 Café  0.09
2                  Spa  0.06
3          Coffee Shop  0.06
4          Pizza Place  0.04


----FYVE DERBYSHIRE----
        

                 venue  freq
0          Coffee Shop  0.12
1     Asian Restaurant  0.12
2  Japanese Restaurant  0.06
3                Trail  0.06
4           Food Court  0.06


----REFLECTIONS AT KEPPEL BAY----
          venue  freq
0      Bus Stop   0.4
1          Park   0.2
2   Pizza Place   0.2
3         Trail   0.2
4  Noodle House   0.0


----REGENT HEIGHTS----
                venue  freq
0  Seafood Restaurant  0.23
1               Beach  0.09
2    Asian Restaurant  0.09
3         Yoga Studio  0.05
4          Water Park  0.05


----RIO VISTA----
         venue  freq
0  Flower Shop  0.12
1  Coffee Shop  0.12
2     Bus Line  0.12
3        Plaza  0.12
4   Playground  0.12


----RIPPLE BAY----
                venue  freq
0         Coffee Shop  0.12
1  Chinese Restaurant  0.12
2                 Gym  0.12
3         Bus Station  0.08
4  Italian Restaurant  0.04


----RIVERFRONT RESIDENCES----
                 venue  freq
0   Chinese Restaurant  0.08
1         Noodle House  0.06
2       Ice

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [66]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [67]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = SG_grouped['Neighborhood']

for ind in np.arange(SG_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(SG_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,134 Bishan Street 12,Chinese Restaurant,Noodle House,Coffee Shop,Fast Food Restaurant,Bakery,Sandwich Place,Seafood Restaurant,Dessert Shop,Asian Restaurant,Food Court
1,141 Marsiling Road,Tea Room,Vegetarian / Vegan Restaurant,Coffee Shop,Gourmet Shop,Bakery,Hainan Restaurant,BBQ Joint,Public Art,Café,Shoe Store
2,159 Bishan Street 13,Bus Stop,Pizza Place,Park,Trail,Women's Store,Flea Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fishing Spot
3,189A Rivervale Drive,Bus Stop,Thai Restaurant,Asian Restaurant,Chinese Restaurant,Gym,Coffee Shop,Café,Grocery Store,Convenience Store,Food Court
4,207B Punggol Place,Playground,Convenience Store,Women's Store,Flower Shop,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fishing Spot,Flea Market,Food & Drink Shop


<a id='item4'></a>

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [71]:
# set number of clusters
kclusters = 5

SG_grouped_clustering = SG_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(SG_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 0, 3, 2, 1, 2, 4, 3, 0])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [75]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

SG_merged = SG_property

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
SG_merged = SG_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Name')

SG_merged.head() # check the last columns!

,Name,Postcode,Price,longitude,latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,THE CREST,159017,1.788000e+06,103.820411,1.292695,1,Pool,Coffee Shop,Gym,Bus Station,Canal,Filipino Restaurant,Fish & Chips Shop,Fishing Spot,Flea Market,Flower Shop
1,CARIBBEAN AT KEPPEL BAY,98657,2.999999e+06,103.758984,1.317181,1,Pub,Italian Restaurant,Bus Line,Bus Station,Tennis Court,Japanese Restaurant,French Restaurant,Dive Bar,Discount Store,Food Truck
2,RIVERFRONT RESIDENCES,530344,9.477000e+05,103.897730,1.410047,3,Chinese Restaurant,Noodle House,Clothing Store,Korean Restaurant,Japanese Restaurant,Bus Station,Market,Ice Cream Shop,Coffee Shop,Asian Restaurant
3,TREASURE AT TAMPINES,520118,1.089190e+06,103.873225,1.365660,3,Hotel,Shopping Mall,Japanese Restaurant,Clothing Store,Boutique,Chinese Restaurant,Bubble Tea Shop,Wine Bar,Asian Restaurant,Café
4,Parc Clematis,128793,1.491167e+06,103.855776,1.346814,2,Playground,Food Court,Coffee Shop,Seafood Restaurant,Noodle House,Basketball Court,Theater,Pizza Place,Discount Store,Dive Bar


Finally, let's visualize the resulting clusters

In [79]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(SG_merged['latitude'], SG_merged['longitude'], SG_merged['Name'], SG_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## 5. Examine Clusters

#### Cluster 1

In [80]:
SG_merged.shape[1]

16

In [81]:
list(range(5, SG_merged.shape[1]))

[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [82]:
SG_merged.columns[[1] + list(range(5, SG_merged.shape[1]))]

Index(['Postcode', 'Cluster Labels', '1st Most Common Venue',
       '2nd Most Common Venue', '3rd Most Common Venue',
       '4th Most Common Venue', '5th Most Common Venue',
       '6th Most Common Venue', '7th Most Common Venue',
       '8th Most Common Venue', '9th Most Common Venue',
       '10th Most Common Venue'],
      dtype='object')

In [83]:
SG_merged.loc[SG_merged['Cluster Labels'] == 0, SG_merged.columns[[1] + list(range(5, SG_merged.shape[1]))]]
# first condition is for rows where meet the condition of Cluster Labels
# second condition is for columns where we only want a certain columns to be displayed in the table

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,570159,0,Bus Stop,Pizza Place,Park,Trail,Women's Store,Flea Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fishing Spot
9,560565,0,Bus Stop,Pizza Place,Park,Trail,Women's Store,Flea Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fishing Spot
10,347693,0,Bus Stop,Pizza Place,Park,Trail,Women's Store,Flea Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fishing Spot
14,530713,0,Bus Stop,Pizza Place,Park,Trail,Women's Store,Flea Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fishing Spot
15,530713,0,Bus Stop,Pizza Place,Park,Trail,Women's Store,Flea Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fishing Spot
17,437938,0,Bus Stop,Pizza Place,Park,Trail,Women's Store,Flea Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fishing Spot
18,530713,0,Bus Stop,Hotel Pool,Trail,Pizza Place,Art Gallery,Roof Deck,Park,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop
93,520282,0,Lighthouse,Pizza Place,Park,Bike Rental / Bike Share,Trail,Women's Store,Fishing Spot,Filipino Restaurant,Fish & Chips Shop,Flea Market
94,35011,0,Bus Stop,Hotel Pool,Trail,Pizza Place,Art Gallery,Roof Deck,Park,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop
100,163027,0,Lighthouse,Pizza Place,Park,Bike Rental / Bike Share,Trail,Women's Store,Fishing Spot,Filipino Restaurant,Fish & Chips Shop,Flea Market


#### Cluster 2

In [84]:
SG_merged.loc[SG_merged['Cluster Labels'] == 1, SG_merged.columns[[1] + list(range(5, SG_merged.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,159017,1,Pool,Coffee Shop,Gym,Bus Station,Canal,Filipino Restaurant,Fish & Chips Shop,Fishing Spot,Flea Market,Flower Shop
1,98657,1,Pub,Italian Restaurant,Bus Line,Bus Station,Tennis Court,Japanese Restaurant,French Restaurant,Dive Bar,Discount Store,Food Truck
19,530713,1,Bus Stop,Bus Station,Gym,BBQ Joint,Food Court,Pool,Dive Bar,Dog Run,French Restaurant,Food Truck
29,530713,1,Fast Food Restaurant,Wings Joint,Seafood Restaurant,Food Court,Metro Station,Bus Station,Supermarket,Flea Market,Filipino Restaurant,Fish & Chips Shop
32,530713,1,Bus Station,Coworking Space,Bus Line,Seafood Restaurant,Restaurant,Food Court,Food & Drink Shop,Food,Exhibit,Flower Shop
56,530713,1,Pool,Furniture / Home Store,Bus Line,Convenience Store,Bus Station,Food Court,Coffee Shop,Flower Shop,Filipino Restaurant,Fish & Chips Shop
57,530713,1,General Entertainment,Bus Station,Metro Station,Asian Restaurant,Park,Chinese Restaurant,Women's Store,Filipino Restaurant,Fish & Chips Shop,Fishing Spot
71,530713,1,Bus Station,Tennis Court,Electronics Store,Women's Store,Flower Shop,Filipino Restaurant,Fish & Chips Shop,Fishing Spot,Flea Market,Food
80,98012,1,Bus Station,Nature Preserve,Trail,Food Truck,Park,Athletics & Sports,Filipino Restaurant,Fish & Chips Shop,Fishing Spot,Flea Market
99,787059,1,Pub,Campground,Park,Exhibit,Food Truck,Food Stand,Food Court,Food & Drink Shop,Food,Flower Shop


#### Cluster 3

In [85]:
SG_merged.loc[SG_merged['Cluster Labels'] == 2, SG_merged.columns[[1] + list(range(5, SG_merged.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,128793,2,Playground,Food Court,Coffee Shop,Seafood Restaurant,Noodle House,Basketball Court,Theater,Pizza Place,Discount Store,Dive Bar
72,530713,2,Supermarket,Food Court,Market,Women's Store,Farmers Market,French Restaurant,Food Truck,Food Stand,Food & Drink Shop,Food
91,809578,2,Sculpture Garden,Playground,Burger Joint,Flea Market,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fishing Spot,Women's Store
111,118171,2,Playground,Food Court,Coffee Shop,Seafood Restaurant,Noodle House,Trail,Theater,Pizza Place,Diner,Discount Store
116,22916,2,Playground,Food Court,Women's Store,Exhibit,French Restaurant,Food Truck,Food Stand,Food & Drink Shop,Food,Flower Shop
132,822207,2,Playground,Convenience Store,Women's Store,Flower Shop,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fishing Spot,Flea Market,Food & Drink Shop
135,823259,2,Playground,Spa,Food Court,Soccer Field,Women's Store,Flea Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fishing Spot


#### Cluster 4

In [86]:
SG_merged.loc[SG_merged['Cluster Labels'] == 3, SG_merged.columns[[1] + list(range(5, SG_merged.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,530344,3,Chinese Restaurant,Noodle House,Clothing Store,Korean Restaurant,Japanese Restaurant,Bus Station,Market,Ice Cream Shop,Coffee Shop,Asian Restaurant
3,520118,3,Hotel,Shopping Mall,Japanese Restaurant,Clothing Store,Boutique,Chinese Restaurant,Bubble Tea Shop,Wine Bar,Asian Restaurant,Café
5,400822,3,Chinese Restaurant,Food Court,Bus Station,Indian Restaurant,Theme Park,Café,Flower Shop,Fast Food Restaurant,Board Shop,Club House
6,828678,3,Food Court,Bus Station,Bus Stop,Coffee Shop,Stadium,Steakhouse,Supermarket,Chinese Restaurant,Lottery Retailer,Bowling Alley
7,554336,3,Food Court,Bus Station,Resort,Pool,Bakery,Coffee Shop,Gym,Indian Restaurant,Metro Station,Supermarket
11,570314,3,Park,Bus Stop,Café,Noodle House,Pet Store,Bus Station,Pizza Place,Indian Restaurant,Lake,Trail
12,268832,3,Café,Hotel,Coffee Shop,Japanese Restaurant,Asian Restaurant,Chinese Restaurant,Pizza Place,Noodle House,Electronics Store,Frozen Yogurt Shop
13,268679,3,Café,Japanese Restaurant,Spa,Coffee Shop,Steakhouse,Wine Bar,Pizza Place,Bakery,Bus Stop,Sake Bar
16,530713,3,Café,Hotel,Coffee Shop,Japanese Restaurant,Asian Restaurant,Chinese Restaurant,Pizza Place,Noodle House,Electronics Store,Frozen Yogurt Shop
20,530713,3,Hotel,Japanese Restaurant,Café,Coffee Shop,Clothing Store,Dessert Shop,Lounge,French Restaurant,Bakery,Shopping Mall


#### Cluster 5

In [87]:
SG_merged.loc[SG_merged['Cluster Labels'] == 4, SG_merged.columns[[1] + list(range(5, SG_merged.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
124,572273,4,Bus Stop,Women's Store,Frozen Yogurt Shop,French Restaurant,Food Truck,Food Stand,Food Court,Food & Drink Shop,Food,Flower Shop
144,768865,4,Bus Stop,BBQ Joint,Bus Station,Women's Store,Filipino Restaurant,Fish & Chips Shop,Fishing Spot,Flea Market,Flower Shop,Food & Drink Shop


<a id='item6'></a>

## 6. Conclusions

From the above analysis, we can get the following conclusion:
1. From the first Map of SG plot with property name and price in the label, we can find that undervalued properties is spread all over the island but most of undervalued properties are located at center of SG, like Orchard and north east area like Puggol area. 
2. In the clustering map, we can find that the cluster 4 have almost all the undervalued properties (green color), from the table of "Cluster 4" we can find he common venues in cluster 4 are Restaurant or Coffee Shop or Food court around these area, but less supermarket and park and school. So these areas are good for investors without kids and don't like cook at home (good for young adults).
3. In Cluster 1, the most common venues are Park, Bus Stop,Pizza Place, Park, Trail, Women's Store, Flea Market and Fast Food Restaurant etc. So these places are good for investors who has a family with kids or pets. 
4. In Cluster 2, the most common venues are Pool, Coffee Shop, Gym,	Bus Station and BBQ Joint etc. So these areas are suitable for investors who love working out and party.
5. In Cluster 3, the most common venues are Playground, Food Court and Convenience Store etc. Hence, these areas are good for investors who wants to set up a school.
6. In Cluster 5, there are only 2 properties which don't have typical features. We will regards it as others.